In [ ]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint

from time import time
requests = 0
from IPython.core.display import clear_output
#start_time = time().requests = 0
from warnings import warn

In [ ]:
#pages = [str(i) for i in range(1,50)]
years_url = [str(i) for i in range(2001,2005)]

pages = ["1", "51", "101", "151", "201", "251"]

pages = ["1"]

# Preparing the monitoring of the loop
start_time = time()
requests = 0

Title = []
Year = []
Rating = []
Metascore = []
Votes = []
Revenue = []
Genre = []
Time = []
Director = []
Stars = []
Description = []

# For every year in the interval 2000-2017
for year_url in years_url:

    # For every page in the interval 1-4
    for page in pages:

        # Make a get request
        response = get('http://www.imdb.com/search/title?release_date=' + year_url + '&sort=num_votes,desc&start=' + page)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        
#        print('Request:{}; Frequency: {} requests/s'.format(requests, requests))

        print('Request:{};'.format(requests))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

        # Break the loop if the number of requests is greater than expected
        if requests > 1000:
            warn('Number of requests was greater than expected.')
            break

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # Select all the 50 movie containers from a single page
        movie_container = page_html.find_all('div', class_ = 'lister-item mode-advanced')

        # For every movie of these 50
        for first_movie in movie_container:
            Title.append(first_movie.h3.a.text)
            year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold')
            Year.append(year.text.strip())
            Rating.append(float(first_movie.strong.text))
            try:
                metascore = first_movie.find('span', class_ = 'metascore favorable')
                Metascore.append(int(metascore.text))
            except:
                Metascore.append(np.nan)
        
            movieNumbers = first_movie.find_all('span', attrs = {'name':'nv'})
            if len(movieNumbers) == 2:
                Votes.append(movieNumbers[0].text)
                Revenue.append(movieNumbers[1].text)
            elif len(movieNumbers) == 1:
                Votes.append(movieNumbers[0].text)
                Revenue.append(np.nan)
            else:
                Votes.append(np.nan)
                Revenue.append(np.nan)
            try:
                time = first_movie.find('span', class_ = 'runtime')
                Time.append(time.text)
            except:
                Time.append(np.nan)
            try:
                genre = first_movie.find('span', class_ = 'genre')
                Genre.append(genre.text.strip())
            except:
                Genre.append(np.nan)
                
            movieCast = first_movie.find("p", class_="")
            try:
                cast = movieCast.text.replace("\n","").split('|')
                cast = [x.strip() for x in cast]
                cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
                Director.append(cast[0])
                Stars.append([x.strip() for x in cast[1].split(",")])
            except:
                cast = movieCast.text.replace("\n","").strip()
                Director.append(np.nan)
                Stars.append([x.strip() for x in cast.split(",")])
        
            Description.append(first_movie.find_all("p", class_="text-muted")[-1].text.lstrip())
                
df = pd.DataFrame({'Title':Title,'Year':Year,'Rating':Rating,'Metascore':Metascore,'Votes':Votes,'Revenue':Revenue,'Genre':Genre,'Time':Time, 'Director':\
      Director, 'Star':Stars, 'Description':Description}) 

df.to_csv('imdb_multi.csv', index=False, encoding='utf-8')
print(df)

                                                 Title    Year  Rating  \
0    The Lord of the Rings: The Fellowship of the Ring  (2001)     8.8   
1                                     A Beautiful Mind  (2001)     8.2   
2                                       Monsters, Inc.  (2001)     8.0   
3                                         Donnie Darko  (2001)     8.0   
4                                               Amélie  (2001)     8.3   
..                                                 ...     ...     ...   
195    Lemony Snicket's A Series of Unfortunate Events  (2004)     6.8   
196                                           EuroTrip  (2004)     6.6   
197                          Resident Evil: Apocalypse  (2004)     6.2   
198                                 Alien vs. Predator  (2004)     5.6   
199                  Harold & Kumar Go to White Castle  (2004)     7.1   

     Metascore      Votes   Revenue                         Genre     Time  \
0         92.0  1,552,471  $315.5